In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader

# Definice jednoduché konvoluční sítě pro extrakci reprezentací
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

    def forward(self, x):
        return self.conv(x)

# Klasifikátor (použijeme po nalezení pseudo-labelů)
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.features = FeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)  # 10 tříd pro FashionMNIST
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

# Příprava datasetu a modelu
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Inicializace modelu pro extrakci reprezentací
feature_extractor = FeatureExtractor()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
feature_extractor.to(device)

# Extrahování latentních reprezentací pro celý dataset
def extract_features(dataloader, model):
    model.eval()  # Přepnutí do eval módu
    features = []
    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            outputs = model(images)
            outputs = outputs.view(outputs.size(0), -1)  # Zploštění
            features.append(outputs.cpu())
    return torch.cat(features)

# Extrakce reprezentací z trénovacích dat
features = extract_features(train_loader, feature_extractor)

# Clustering (k-means) na extrahovaných reprezentacích
kmeans = KMeans(n_clusters=10, random_state=0).fit(features)

# Pseudo-labely získané z clusteringu
pseudo_labels = kmeans.labels_

# Vytvoření datasetu s pseudo-labely
class PseudoLabeledDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, pseudo_labels):
        self.dataset = dataset
        self.pseudo_labels = pseudo_labels

    def __getitem__(self, index):
        img, _ = self.dataset[index]
        pseudo_label = self.pseudo_labels[index]
        return img, pseudo_label

    def __len__(self):
        return len(self.dataset)

pseudo_labeled_dataset = PseudoLabeledDataset(train_dataset, pseudo_labels)
pseudo_loader = DataLoader(pseudo_labeled_dataset, batch_size=64, shuffle=True)

# Trénování klasifikátoru na pseudo-labely
classifier = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Trénování klasifikátoru na pseudo-labely
num_epochs = 30

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in pseudo_loader:
        images = images.to(device)
        labels = labels.to(device).long()  # Přidána konverze pseudo-labelů na torch.LongTensor

        # Dopředný průchod
        outputs = classifier(images)
        loss = criterion(outputs, labels)

        # Zpětný průchod a optimalizace
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(pseudo_loader):.4f}')

print('Trénování klasifikátoru s pseudo-labely dokončeno!')

# ------------------------ TESTOVÁNÍ MODELŮ ------------------------

# Testovací data pro vyhodnocení klasifikace
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Funkce pro vyhodnocení přesnosti na testovacích datech
def evaluate_model(dataloader, model):
    model.eval()  # Přepnutí do eval módu (vypnutí dropout/batch norm)
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Vyhodnocení přesnosti na testovací sadě
test_accuracy = evaluate_model(test_loader, classifier)
print(f'Test Accuracy: {test_accuracy:.2f}%')


Epoch [1/30], Loss: 2.2491
Epoch [2/30], Loss: 2.2462
Epoch [3/30], Loss: 2.2454
Epoch [4/30], Loss: 2.2450
Epoch [5/30], Loss: 2.2434
Epoch [6/30], Loss: 2.2409
Epoch [7/30], Loss: 2.2354
Epoch [8/30], Loss: 2.2256
Epoch [9/30], Loss: 2.2086
Epoch [10/30], Loss: 2.1800
Epoch [11/30], Loss: 2.1396
Epoch [12/30], Loss: 2.0882
Epoch [13/30], Loss: 2.0255
Epoch [14/30], Loss: 1.9537
Epoch [15/30], Loss: 1.8781
Epoch [16/30], Loss: 1.8003
Epoch [17/30], Loss: 1.7209
Epoch [18/30], Loss: 1.6434
Epoch [19/30], Loss: 1.5708
Epoch [20/30], Loss: 1.4988
Epoch [21/30], Loss: 1.4303
Epoch [22/30], Loss: 1.3641
Epoch [23/30], Loss: 1.3035
Epoch [24/30], Loss: 1.2467
Epoch [25/30], Loss: 1.1918
Epoch [26/30], Loss: 1.1423
Epoch [27/30], Loss: 1.0936
Epoch [28/30], Loss: 1.0529
Epoch [29/30], Loss: 1.0112
Epoch [30/30], Loss: 0.9695
Trénování klasifikátoru s pseudo-labely dokončeno!
Test Accuracy: 9.98%
